In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Current Device:", torch.cuda.current_device())
print("Device Name:", torch.cuda.get_device_name(0))

CUDA Available: True
Current Device: 0
Device Name: NVIDIA GeForce RTX 4060 Laptop GPU


## Load Required Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torch.optim.lr_scheduler import CosineAnnealingLR
import os
import random
import time
import copy
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

## Load & Preprocess IP102 Dataset

# ResNet50:

In [3]:
# Dataset Path
dataset_path = r'C:\Users\merye\Desktop\IP102_Wheat_Cereal_Pests'

In [4]:
# Data transformation for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

# Paths and loading data
data_dir = 'C:/Users/merye/Desktop/IP102_Wheat_Cereal_Pests'
image_datasets = {x: datasets.ImageFolder(root=f"{data_dir}/{x}", transform=data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

# Device setup (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ResNet50 model with customized fully connected layer
def create_model():
    model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Dropout(0.6),
        nn.Linear(num_ftrs, num_classes)
    )
    return model.to(device)

# Training loop with early stopping
def train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=3):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}\n' + '-' * 50)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss, running_corrects = 0.0, 0
            start_time = time.time()

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            elapsed_time = time.time() - start_time

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} | Time: {elapsed_time:.2f}s')

            if phase == 'val':
                scheduler.step()

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_counter = 0
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= patience:
                        print("Early stopping triggered! Training halted.")
                        model.load_state_dict(best_model_wts)
                        return model

    model.load_state_dict(best_model_wts)
    return model

# Initialize model, loss, optimizer, and scheduler
model = create_model()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-6)

# Train model
model = train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=3)

# Save model
torch.save(model.state_dict(), "resnet50_finetuned.pth")
print("Model training complete and saved!")


Epoch 1/30
--------------------------------------------------
Train Loss: 2.1917 | Acc: 0.3995 | Time: 228.36s
Val Loss: 1.6316 | Acc: 0.6005 | Time: 24.43s
Epoch 2/30
--------------------------------------------------
Train Loss: 1.6073 | Acc: 0.6141 | Time: 161.14s
Val Loss: 1.3160 | Acc: 0.7185 | Time: 20.63s
Epoch 3/30
--------------------------------------------------
Train Loss: 1.4071 | Acc: 0.6774 | Time: 210.15s
Val Loss: 1.2270 | Acc: 0.7493 | Time: 21.65s
Epoch 4/30
--------------------------------------------------
Train Loss: 1.3135 | Acc: 0.7216 | Time: 225.42s
Val Loss: 1.2424 | Acc: 0.7265 | Time: 19.93s
Epoch 5/30
--------------------------------------------------
Train Loss: 1.1921 | Acc: 0.7679 | Time: 222.60s
Val Loss: 1.1358 | Acc: 0.7922 | Time: 21.72s
Epoch 6/30
--------------------------------------------------
Train Loss: 1.1231 | Acc: 0.7942 | Time: 220.98s
Val Loss: 1.1308 | Acc: 0.7788 | Time: 17.79s
Epoch 7/30
-----------------------------------------------

# EfficientB2:

In [5]:
# Load Pretrained EfficientNetB2 and Modify
def create_model():
    model = models.efficientnet_b2(weights="DEFAULT")  # EfficientNetB2
    num_ftrs = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(0.6),
        nn.Linear(num_ftrs, num_classes)
    )
    return model.to(device)

# Training function with Early Stopping
def train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=5):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    early_stop_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}\n' + '-' * 50)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss, running_corrects = 0.0, 0
            start_time = time.time()

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            elapsed_time = time.time() - start_time

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} | Acc: {epoch_acc:.4f} | Time: {elapsed_time:.2f}s')

            if phase == 'val':
                scheduler.step()

                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_counter = 0  # Reset early stopping
                else:
                    early_stop_counter += 1
                    if early_stop_counter >= patience:
                        print("Early stopping triggered! Best model restored.")
                        model.load_state_dict(best_model_wts)
                        return model

    model.load_state_dict(best_model_wts)
    return model

# Initialize model, loss, optimizer, and scheduler
model = create_model()

# Loss function with label smoothing and CrossEntropy
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# Optimizer with reduced learning rate and weight decay
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=5e-5)

# Cosine Annealing scheduler with adjusted training duration
scheduler = CosineAnnealingLR(optimizer, T_max=30, eta_min=1e-6)

# Train model with early stopping
model = train_model(model, criterion, optimizer, scheduler, num_epochs=30, patience=5)

# Save model
torch.save(model.state_dict(), "efficientnet_b2_finetuned.pth")
print("Model training complete and saved!")


Epoch 1/30
--------------------------------------------------
Train Loss: 2.6070 | Acc: 0.2347 | Time: 42.54s
Val Loss: 2.0817 | Acc: 0.4718 | Time: 8.40s
Epoch 2/30
--------------------------------------------------
Train Loss: 2.0432 | Acc: 0.4672 | Time: 42.42s
Val Loss: 1.5993 | Acc: 0.6582 | Time: 6.67s
Epoch 3/30
--------------------------------------------------
Train Loss: 1.7280 | Acc: 0.5716 | Time: 39.96s
Val Loss: 1.3494 | Acc: 0.7185 | Time: 7.00s
Epoch 4/30
--------------------------------------------------
Train Loss: 1.5279 | Acc: 0.6438 | Time: 53.71s
Val Loss: 1.2479 | Acc: 0.7359 | Time: 7.54s
Epoch 5/30
--------------------------------------------------
Train Loss: 1.4051 | Acc: 0.6919 | Time: 79.03s
Val Loss: 1.1816 | Acc: 0.7694 | Time: 8.01s
Epoch 6/30
--------------------------------------------------
Train Loss: 1.3250 | Acc: 0.7274 | Time: 65.37s
Val Loss: 1.1343 | Acc: 0.7922 | Time: 7.51s
Epoch 7/30
--------------------------------------------------
Train Lo